CIS-PD Form Data

Use datadictionary and codelist to convert the scores on each form.
Save each file in a pickle format.

In [2]:
import os
import platform
import pandas as pd
import pickle #to save files

In [5]:
## modify paths
if platform.system() == 'Windows':
    if platform.release() == '7':
        path = r'Y:\CIS-PD MUSC'
        #folder_path = r'Y:\Inpatient Sensors -Stroke\Data\biostamp_data'
        dict_path = r'Y:\CIS-PD MUSC\Data_dict'

In [ ]:
#complete = list(['Heart Rate Variability', 'MDS-UPDRS #1: Finger Tapping'])
def process_annotations(path):
#---------------------------------------------------------------------------------------------------------
# Processes raw annotations file to extract start / end timestamps and remove unnecessary data
#
# Inputs:  path - filepath of the subject folder containing annotations.csv
#
# Outputs: df - dataframe containing list of activities and their start / end timestamps
#---------------------------------------------------------------------------------------------------------
    df = pd.read_csv(os.path.join(path, 'annotations.csv'))
    del df['Timestamp (ms)']
    del df['AnnotationId']
    del df['AuthorId']
    del df['Value']
    
    df = df[(df.EventType != 'Testing Day')]
    
    sorter = set(df.EventType.unique().flatten())
    sorterIndex = dict(zip(sorter, range(len(sorter))))
        
    df['EventType_Rank'] = df['EventType'].map(sorterIndex)
    df['Cycle'] = df.groupby('EventType')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
    del df['EventType_Rank']
    df[df['EventType'].str.contains('Heart')] = df[df['EventType'].str.contains('Heart')].replace(to_replace={'Cycle': {1: 'NaN', 2: 'NaN', 3: 'NaN', 4: 'NaN'}})
    df = df.reset_index(drop=True).set_index('EventType')
    
    return df

In [ ]:
# modify this code
def extract_data(SubID, path):
    timestamps = process_annotations(path)
    ....
    for filenames in files:
        if filenames.startswith('form'):
            p = pathlib.Path(os.path.join(root, filenames))
            location = str(p.relative_to(path)).split("\\")[0]
            temp_df = pd.read_csv(p).set_index('Timestamp (ms)')
            accel[location] = accel[location].append(temp_df)
    complete_acts = complete
    # Complete dictionary of all activities
    act_dict = {acts: pd.DataFrame() for acts in complete_acts}
    return act_dict, timestamps

In [6]:
dd = pd.read_sas('Y:\CIS-PD MUSC\datadictionary.sas7bdat')
print(dd.shape)
dd.head(10)

(1221, 11)


,zFormID,zCRFNb,zFieldNb,zFieldNm,zSASNm,zSASLabel,zFieldTypeNm,zLength,zUnit,zRequired,zCodeGroup
0,101.0,b'101',b'zConsentDate',b'Date of signed informed consent',NaN,NaN,b'Date',0.0,NaN,0.0,NaN
1,101.0,b'101',b'zAge',b'Age',NaN,NaN,b'Number',0.0,NaN,0.0,NaN
2,101.0,b'101',b'Q03',b'Motor fluctuations believed to be on average...,NaN,NaN,b'Radio button',0.0,NaN,2.0,51.0
3,101.0,b'101',b'Q04',b'Diagnosis of idiopathic Parkinson\x92s disease',NaN,NaN,b'Radio button',0.0,NaN,0.0,51.0
4,101.0,b'101',b'Q05',b'Hoehn & Yahr stage 1-3',NaN,NaN,b'Radio button',0.0,NaN,0.0,51.0
5,101.0,b'101',b'Q06',b'Ability to travel to a study site to attend ...,NaN,NaN,b'Radio button',0.0,NaN,0.0,51.0
6,101.0,b'101',b'Q07',b'Ability to read and understand English',NaN,NaN,b'Radio button',0.0,NaN,0.0,51.0
7,101.0,b'101',b'Q08',b'Owns and is the sole user of an iPhone 5s an...,NaN,NaN,b'Radio button',0.0,NaN,0.0,51.0
8,101.0,b'101',b'Q11',b'iPhone model to be used by subject',NaN,NaN,b'Text',50.0,NaN,4.0,NaN
9,101.0,b'101',b'Q12',b'iOS version to be used',NaN,NaN,b'Text',50.0,NaN,4.0,NaN


In [7]:
codelist = pd.read_sas('Y:\CIS-PD MUSC\codelist.sas7bdat')
print(codelist.shape)
print(codelist.head(5))
print(codelist.tail(5))

(951, 3)
   zGroupID  zItemNb                                            zItemNm
0       1.0      1.0  b'Global non-project related tables, such as z...
1       1.0      2.0  b'Global project related study design tables, ...
2       1.0      3.0  b'Global project related form setup tables, su...
3       1.0      4.0  b'Subject related No CRF tables, such as subje...
4       1.0      5.0  b'CRF related no CRF tables, such as subject C...
     zGroupID  zItemNb                                            zItemNm
946     589.0      4.0  b'Associates degree \r\n        (including occ...
947     589.0      5.0              b'Bachelors degree (BA, BS, AB, etc)'
948     589.0      6.0         b'Masters degree (MA, MS, MENG, MSW, etc)'
949     589.0      7.0    b'Professional school degree (MD, DDC, JD etc)'
950     589.0      8.0                b'Doctorate degree (PhD, EdD, etc)'


In [8]:
form101 = pd.read_sas('Y:\CIS-PD MUSC\\form101.sas7bdat')
print(form101.shape)
form101.head(10)

(56, 15)


,zSubjectCode,SiteID,zVisitNm,F101Q11,F101Q12,F101zConsentDate,F101zAge,F101Q03,F101Q04,F101Q05,F101Q06,F101Q07,F101Q08,F101Q09,F101Q10
0,1000.0,1313.0,b'Baseline',b'A1784',b'10.3.2',2017-06-15,63.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,1001.0,1313.0,b'Baseline',b'A1778',b'10.3.2',2017-06-15,64.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,1002.0,1313.0,b'Baseline',b'A1586',b'10.3.2',2017-06-16,51.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3,1003.0,1313.0,b'Baseline',b'A1522',b'10.3.2',2017-06-19,60.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,1004.0,1332.0,b'Baseline',b'A1660',b'10.3.2',2017-06-21,52.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
5,1005.0,1313.0,b'Baseline',b'A1661',b'10.3.1',2017-06-21,52.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
6,1006.0,1396.0,b'Baseline',b'A1533',b'11.0.2 (15A421)',2017-06-21,71.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
7,1007.0,1313.0,b'Baseline',b'A1778',b'10.3.2',2017-06-23,75.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
8,1008.0,1396.0,b'Baseline',b'NA',b'NA',2017-06-23,86.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
9,1009.0,1313.0,b'Baseline',b'A1522',b'10.3.2',2017-06-26,54.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [3]:
######## play
#############
d1 = {'Q03': [1.0, 0.0,1.0], 'Q04': [0.0, 1.0, 0.0]}
form101 = pd.DataFrame(data=d1)
d2 = {'FormID': [101.0, 101.0], 'ItemNb': [0.0, 1.0], 'ItemNm': ['no','yes']}
result = pd.DataFrame(data=d2)


In [ ]:
# subset result to match FormID
# test = result[(result.FormID==101.0)]

In [6]:
print(form101)
print(result)

   Q03  Q04
0  1.0  0.0
1  0.0  1.0
2  1.0  0.0
   FormID  ItemNb ItemNm
0   101.0     0.0     no
1   101.0     1.0    yes


In [ ]:
# I need to merge it now
#join with key of Q03 with ItemNb # but need to change values so NO
#update # doesn't seem to work either
form101.merge(result, left_on='Q03', right_on='ItemNb', how='outer')


In [ ]:
# use dict N keys to 1 value



In [ ]:
# https://stackoverflow.com/questions/14692690/access-nested-dictionary-items-via-a-list-of-keys

In [10]:
#form101 = pd.read_sas('Y:\CIS-PD MUSC\\form101.sas7bdat')
form126 = pd.read_sas('Y:\CIS-PD MUSC\\form126.sas7bdat')
form238 = pd.read_sas('Y:\CIS-PD MUSC\\form238.sas7bdat')
form239 = pd.read_sas('Y:\CIS-PD MUSC\\form239.sas7bdat')
form240 = pd.read_sas('Y:\CIS-PD MUSC\\form240.sas7bdat')
form241 = pd.read_sas('Y:\CIS-PD MUSC\\form241.sas7bdat')
form242 = pd.read_sas('Y:\CIS-PD MUSC\\form242.sas7bdat')
form243 = pd.read_sas('Y:\CIS-PD MUSC\\form243.sas7bdat')
form501 = pd.read_sas('Y:\CIS-PD MUSC\\form501.sas7bdat')
form502 = pd.read_sas('Y:\CIS-PD MUSC\\form502.sas7bdat')
form503 = pd.read_sas('Y:\CIS-PD MUSC\\form503.sas7bdat')
form504 = pd.read_sas('Y:\CIS-PD MUSC\\form504.sas7bdat')
form505 = pd.read_sas('Y:\CIS-PD MUSC\\form505.sas7bdat')
form506 = pd.read_sas('Y:\CIS-PD MUSC\\form506.sas7bdat')
form508 = pd.read_sas('Y:\CIS-PD MUSC\\form508.sas7bdat')
form509 = pd.read_sas('Y:\CIS-PD MUSC\\form509.sas7bdat')
form510 = pd.read_sas('Y:\CIS-PD MUSC\\form510.sas7bdat')
form515 = pd.read_sas('Y:\CIS-PD MUSC\\form515.sas7bdat')
form518 = pd.read_sas('Y:\CIS-PD MUSC\\form518.sas7bdat')

In [ ]:
# Create dictionaries for all forms
d = os.listdir(path)
f = [filename for filename in d if filename.startswith('form')] #need to update to skip existing files in /data
print(f)

In [ ]:
#create data dict for remaining subjects
data_all = []
for FormID in list(set(f)):
    print('Loading Form ' + FormID + ' Data...')
    act_dict, timestamps = extract_data(FormID, os.path.join(path, FormID))
    print('Extract data complete.')
    #save dict to Pickle file
    filename = dict_path+'\\'+FormID + 'dict.pkl'
    filename = os.path.join(dict_path, FormID + 'dict.pkl')
    print(filename)
    f = open(filename,'wb')
    pickle.dump(act_dict,f)
    f.close()
    print(filename + ' ' + 'File Saved\n')

In [ ]:
print(os.listdir(dict_path))
print(len(os.listdir(dict_path)))